# 🚢 Kaggle Competition: Titanic - Machine Learning from Disaster
**Notebook: EPA Titanic KNN v1.1**

## 🎯 Objetivo
Predecir qué pasajeros sobrevivieron al hundimiento del Titanic usando un modelo de **K-Nearest Neighbors (KNN)**.  
Este notebook busca ser un ejercicio de aprendizaje, mostrando paso a paso cómo preparar los datos, entrenar un modelo, ajustar hiperparámetros y generar un archivo `submission.csv` válido para Kaggle.

## 📂 Dataset
Los archivos provienen de la competición oficial en Kaggle:
- **train.csv** → 891 pasajeros, con la variable objetivo `Survived`.
- **test.csv** → 418 pasajeros, sin `Survived` (a predecir).
- **gender_submission.csv** → ejemplo de submission válido.

## 🧩 Flujo del Notebook
1. ⚙️ Configuración inicial de Kaggle  
2. 📥 Carga de datos (train y test)  
3. 🔎 Análisis exploratorio inicial (faltantes y balance de clases)  
4. 🧱 Selección de variables (features y objetivo)  
5. 🧽 Definición del preprocesamiento (imputación, escalado, one-hot)  
6. 🤖 Pipeline con KNN y validación cruzada  
7. 🔎 Búsqueda de hiperparámetros (Grid Search para KNN)  
8. 📤 Entrenamiento final y creación del archivo `submission.csv`

---

✨ **Nota:** Todo el notebook está comentado en español, para facilitar la comprensión del flujo de trabajo en Kaggle.

# ⚙️ Celda 1 — Configuración inicial de Kaggle
Este bloque viene por defecto en el notebook. Lista los archivos disponibles en `/kaggle/input/titanic/`.

In [11]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


# 📥 Celda 2 — Carga de datos (train y test)
Leemos los archivos `train.csv` y `test.csv` y verificamos sus dimensiones.

In [8]:
# Importamos pandas para leer CSV y manipular DataFrames
import pandas as pd

# Definimos las rutas de los archivos provistos por la competencia
train_path = "/kaggle/input/titanic/train.csv"   # ruta del set de entrenamiento
test_path  = "/kaggle/input/titanic/test.csv"    # ruta del set de test (sin Survived)

# Leemos los CSV a DataFrames de pandas
train_df = pd.read_csv(train_path)  # carga train.csv
test_df  = pd.read_csv(test_path)   # carga test.csv

# Inspeccionamos las dimensiones de cada DataFrame
print("Shape train:", train_df.shape)  # imprime filas/columnas de train
print("Shape test :", test_df.shape)   # imprime filas/columnas de test

# Mostramos las primeras filas del set de entrenamiento para conocer la estructura
train_df.head()  # vista rápida de columnas y ejemplos


Shape train: (891, 12)
Shape test : (418, 11)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


# 🔎 Celda 3 — Análisis exploratorio inicial
Revisamos columnas disponibles, valores faltantes en cada dataset y el balance de la variable objetivo `Survived`.

In [9]:
# Mostramos las columnas disponibles en el dataset de entrenamiento
print("Columnas en train:", train_df.columns.tolist())

# Revisamos cuántos valores faltantes (NaN) hay por columna en train
print("\nFaltantes en train:")
print(train_df.isna().sum())

# Revisamos cuántos valores faltantes (NaN) hay por columna en test
print("\nFaltantes en test:")
print(test_df.isna().sum())

# Verificamos la distribución de la variable objetivo Survived (0 = no, 1 = sí)
print("\nDistribución de Survived en train:")
print(train_df["Survived"].value_counts(normalize=True))


Columnas en train: ['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']

Faltantes en train:
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

Faltantes en test:
PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

Distribución de Survived en train:
Survived
0    0.616162
1    0.383838
Name: proportion, dtype: float64


# 🧱 Celda 4 — Selección de variables (features y objetivo)
Definimos qué columnas usaremos como variables predictoras (`X`) y cuál será la variable objetivo (`y`).

In [10]:
# Definimos las columnas numéricas que usaremos como características (features)
num_features = ["Age", "SibSp", "Parch", "Fare", "Pclass"]  
# -> Edad, nº de hermanos/esposos, nº de padres/hijos, tarifa y clase social

# Definimos las columnas categóricas que usaremos como características
cat_features = ["Sex", "Embarked"]  
# -> Sexo y puerto de embarque

# Construimos la matriz de características X a partir de train
X = train_df[num_features + cat_features]

# Construimos el vector objetivo y (sobrevivió o no)
y = train_df["Survived"].astype(int)

# Guardamos los PassengerId de test para poder armar el submission al final
test_passenger_id = test_df["PassengerId"].copy()

# Construimos la matriz de características X_test con las mismas columnas que X
X_test = test_df[num_features + cat_features]

# Revisamos las primeras filas de X para validar que tenemos las features correctas
X.head()


,Age,SibSp,Parch,Fare,Pclass,Sex,Embarked
0,22.0,1,0,7.2500,3,male,S
1,38.0,1,0,71.2833,1,female,C
2,26.0,0,0,7.9250,3,female,S
3,35.0,1,0,53.1000,1,female,S
4,35.0,0,0,8.0500,3,male,S


# 🧽 Celda 5 — Definición del preprocesamiento
Creamos pipelines de preprocesamiento para variables numéricas (imputación + escalado) y categóricas (imputación + one-hot).

In [6]:
# Importamos los módulos necesarios para preprocesar y construir pipelines
from sklearn.compose import ColumnTransformer                 # permite aplicar transformaciones por tipo de columna
from sklearn.pipeline import Pipeline                         # encadena pasos (preprocesamiento + modelo)
from sklearn.impute import SimpleImputer                      # para imputar (rellenar) valores faltantes
from sklearn.preprocessing import OneHotEncoder, StandardScaler  # codificación categórica y escalado numérico

# Definimos un pipeline para columnas numéricas
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),  # rellena NaN con la mediana (robusta a outliers)
    ("scaler", StandardScaler())                    # estandariza a media 0 y desvío 1 (crítico para KNN)
])

# Definimos un pipeline para columnas categóricas
categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),          # rellena NaN con la moda (valor más frecuente)
    ("onehot", OneHotEncoder(handle_unknown="ignore", sparse=False))  # convierte categorías a variables dummies
])

# Combinamos ambos pipelines por tipo de columna usando ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, num_features),  # aplica el pipeline numérico a las columnas numéricas
        ("cat", categorical_transformer, cat_features)  # aplica el pipeline categórico a las columnas categóricas
    ],
    remainder="drop"  # descarta cualquier columna no especificada (por seguridad)
)

# vista rápida: mostramos qué columnas estamos declarando por tipo
print("Columnas numéricas:", num_features)    # debería listar ['Age', 'SibSp', 'Parch', 'Fare', 'Pclass']
print("Columnas categóricas:", cat_features)  # debería listar ['Sex', 'Embarked']


Columnas numéricas: ['Age', 'SibSp', 'Parch', 'Fare', 'Pclass']
Columnas categóricas: ['Sex', 'Embarked']


# 🩹 Celda 5.1 — Parche OneHotEncoder (evitar FutureWarning)
Re-definimos el `OneHotEncoder` usando el nuevo parámetro `sparse_output=False` para eliminar las advertencias de futuro de scikit-learn.

In [13]:
# 🩹 Parche: re-definimos el transformador categórico usando el parámetro moderno 'sparse_output'
from sklearn.preprocessing import OneHotEncoder

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),                 # moda para NaN
    ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=False))  # dummies densos sin warning
])

# Re-construimos el preprocessor con el nuevo categorical_transformer
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, num_features),   # numéricas: mediana + escalado
        ("cat", categorical_transformer, cat_features) # categóricas: moda + one-hot
    ],
    remainder="drop"
)

print("✅ Parche aplicado: OneHotEncoder usa 'sparse_output=False'")


✅ Parche aplicado: OneHotEncoder usa 'sparse_output=False'


# 🤖 Celda 6 — Pipeline con KNN y validación cruzada
Armamos un pipeline que incluye el preprocesamiento y un modelo KNN. Evaluamos su desempeño con validación cruzada estratificada.i

In [14]:
# Importamos el clasificador KNN de scikit-learn
from sklearn.neighbors import KNeighborsClassifier
# Importamos validación cruzada estratificada (mantiene proporción de clases en cada fold)
from sklearn.model_selection import StratifiedKFold, cross_val_score
# Importamos numpy para operaciones numéricas y redondeos
import numpy as np

# Definimos un modelo base de KNN con hiperparámetros razonables de partida
base_knn = KNeighborsClassifier(
    n_neighbors=7,   # número de vecinos (k); valor inicial que suele rendir bien en Titanic
    weights="uniform",  # todos los vecinos pesan igual (alternativa: "distance")
    p=2                # métrica Minkowski con p=2 equivale a distancia Euclídea
)

# Construimos el pipeline completo: primero preprocesa, luego entrena el KNN
pipe_knn = Pipeline(steps=[
    ("preprocess", preprocessor),  # aplica imputación, escalado y one-hot definidos en la Celda 5
    ("knn", base_knn)              # clasificador KNN como último paso
])

# Definimos la estrategia de validación cruzada estratificada con 5 pliegues
cv = StratifiedKFold(
    n_splits=5,     # número de folds
    shuffle=True,   # baraja los datos antes de dividir
    random_state=42 # semilla para reproducibilidad
)

# Ejecutamos la validación cruzada usando accuracy como métrica
cv_scores = cross_val_score(
    estimator=pipe_knn,  # el pipeline completo (prepro + modelo)
    X=X,                 # matriz de características de entrenamiento (de la Celda 4)
    y=y,                 # vector objetivo (de la Celda 4)
    cv=cv,               # esquema de validación cruzada
    scoring="accuracy",  # métrica de evaluación
    n_jobs=-1            # usa todos los núcleos disponibles para acelerar
)

# Imprimimos los resultados de accuracy por fold redondeados a 4 decimales
print("Accuracy por fold:", np.round(cv_scores, 4))
# Imprimimos el promedio y la desviación estándar de accuracy en los 5 folds
print("Accuracy promedio (CV):", cv_scores.mean().round(4), "| Desv. std:", cv_scores.std().round(4))


Accuracy por fold: [0.8156 0.809  0.7978 0.8146 0.8202]
Accuracy promedio (CV): 0.8114 | Desv. std: 0.0077


# 🔎 Celda 7 — Búsqueda de hiperparámetros (Grid Search para KNN)
Usamos `GridSearchCV` para encontrar la mejor combinación de `n_neighbors`, `weights` y `p` (distancia), manteniendo el preprocesamiento dentro del pipeline.

In [15]:
# Importamos las utilidades necesarias para la búsqueda en grilla
from sklearn.model_selection import GridSearchCV                 # búsqueda exhaustiva de hiperparámetros
from sklearn.neighbors import KNeighborsClassifier               # clasificador KNN
from sklearn.model_selection import StratifiedKFold              # CV estratificada (para reproducibilidad)
import pandas as pd                                              # para tabular resultados

# Reconstruimos el pipeline (por claridad) con el preprocesador ya definido y un KNN "vacío" (lo definirá la grilla)
pipe_knn = Pipeline(steps=[
    ("preprocess", preprocessor),                                # imputación + escalado + one-hot
    ("knn", KNeighborsClassifier())                              # modelo KNN sin hiperparámetros fijos
])

# Definimos una grilla razonable de hiperparámetros para Titanic
param_grid = {
    "knn__n_neighbors": [3, 5, 7, 9, 11, 13, 15, 19, 25],       # distintos valores de k
    "knn__weights": ["uniform", "distance"],                     # vecinos con peso igual o ponderado por distancia
    "knn__p": [1, 2]                                             # p=1 (Manhattan), p=2 (Euclídea)
}

# Reutilizamos la misma validación cruzada estratificada (5 folds, barajada, semilla fija)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Configuramos el GridSearchCV
grid = GridSearchCV(
    estimator=pipe_knn,                                          # pipeline completo
    param_grid=param_grid,                                       # grilla de hiperparámetros
    scoring="accuracy",                                          # métrica de evaluación (oficial de la comp)
    cv=cv,                                                       # esquema de validación
    n_jobs=-1,                                                   # usar todos los cores disponibles
    refit=True,                                                  # reentrena el mejor modelo con todo el train
    verbose=1                                                    # mostrar progreso por consola
)

# Ejecutamos la búsqueda (entrena un modelo por cada combinación de la grilla)
grid.fit(X, y)                                                   # ajusta sobre todo el train

# Mostramos el mejor accuracy promedio en CV y la mejor combinación de hiperparámetros
print("🔝 Mejor accuracy (CV):", grid.best_score_.round(4))       # mejor performance promedio
print("🔧 Mejores hiperparámetros:", grid.best_params_)           # diccionario con los mejores params

# Guardamos el mejor estimador ya reentrenado (pipeline óptimo listo para predecir)
best_model = grid.best_estimator_

# Tabulamos los 5 mejores resultados para inspección rápida
results = pd.DataFrame(grid.cv_results_).sort_values("mean_test_score", ascending=False)
cols = ["mean_test_score", "std_test_score", "param_knn__n_neighbors", "param_knn__weights", "param_knn__p"]
try:
    display(results[cols].head(5))                               # display si está disponible (Kaggle/Notebook)
except NameError:
    print(results[cols].head(5))                                  # fallback a print si no existe display()


Fitting 5 folds for each of 36 candidates, totalling 180 fits
🔝 Mejor accuracy (CV): 0.8238
🔧 Mejores hiperparámetros: {'knn__n_neighbors': 11, 'knn__p': 1, 'knn__weights': 'uniform'}


,mean_test_score,std_test_score,param_knn__n_neighbors,param_knn__weights,param_knn__p
16,0.823803,0.015207,11,uniform,1
8,0.822666,0.012162,7,uniform,1
12,0.820438,0.011587,9,uniform,1
20,0.815937,0.011984,13,uniform,1
24,0.814808,0.008096,15,uniform,1


# 📤 Celda 8 — Entrenamiento final y creación de `submission.csv`
Usamos el **mejor pipeline** encontrado por GridSearchCV (`best_model`) para predecir en `test.csv` y generamos el archivo de envío con las columnas `PassengerId` y `Survived`.

In [16]:
# best_model ya quedó entrenado (refit=True) sobre todo el train al finalizar el Grid Search

# 1) Predicciones sobre el set de test
test_preds = best_model.predict(X_test)        # obtiene 0/1 por pasajero
test_preds = test_preds.astype(int)            # aseguramos que sean enteros

# 2) Construimos el DataFrame de submission (exactamente 2 columnas)
submission = pd.DataFrame({
    "PassengerId": test_passenger_id,          # IDs del test en el mismo orden
    "Survived": test_preds                     # predicciones binarias
})

# chequeo rápido de formato
assert submission.shape[0] == 418, "El submission debe tener exactamente 418 filas."
assert list(submission.columns) == ["PassengerId", "Survived"], "El submission debe tener dos columnas exactas."

# 3) Guardamos el CSV listo para subir a Kaggle
submission_file = "submission.csv"
submission.to_csv(submission_file, index=False)

print(f"✅ Archivo '{submission_file}' creado con {submission.shape[0]} filas.")
print(submission.head())
print("\nConteo predicciones:", submission['Survived'].value_counts().to_dict())


✅ Archivo 'submission.csv' creado con 418 filas.
   PassengerId  Survived
0          892         0
1          893         0
2          894         0
3          895         0
4          896         1

Conteo predicciones: {0: 263, 1: 155}


# 🏁 Resultado del Submission

**Latest Score:** `0.76315` 

✅ Este notebook corresponde a la versión **V2 (KNN con preprocesamiento + Grid Search)**.  
